# Import Lib

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
import os
import time

# Set required values ​​and paths

In [ ]:
# --- 1. กำหนดค่าและ Path ที่จำเป็น ---

# ## แก้ไขตรงนี้ ##
# Path ไปยังชุดข้อมูลของคุณ (ต้องเป็น Path เดียวกับที่ใช้ใน Notebook)
datapath = '/media/capybara/Data/dataset_vit/archive'

# ## แก้ไขตรงนี้ ##
# Path ไปยังไฟล์โมเดล .keras ที่ดีที่สุดที่คุณบันทึกไว้
best_model_path = '/media/capybara/Data/dataset_vit/resnet50/best_resnet_model_imagenet2012v2_classes.keras'

# พารามิเตอร์ของรูปภาพ (ต้องตรงกับตอนเทรน)
img_height = 224
img_width = 224
batch_size = 32

# Load model and Validation dataset

In [ ]:
# --- 2. โหลดโมเดลและชุดข้อมูล Validation ---

print(f"กำลังโหลดโมเดลจาก: {best_model_path}")
if not os.path.exists(best_model_path):
    raise FileNotFoundError(f"ไม่พบไฟล์โมเดลที่ '{best_model_path}'. กรุณาตรวจสอบ Path ให้ถูกต้อง")
    
model = tf.keras.models.load_model(best_model_path)
print("โหลดโมเดลสำเร็จ")

from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input

print(f"กำลังโหลดชุดข้อมูล Validation จาก: {datapath}")
val_dataset = tf.keras.utils.image_dataset_from_directory(
    datapath,
    validation_split=0.1,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False
)

class_names = val_dataset.class_names
print(f"พบ {len(class_names)} คลาส")

def preprocess_val_data(images, labels):
    images = resnet_preprocess_input(images)
    return images, labels

val_pipeline = val_dataset.map(preprocess_val_data, num_parallel_calls=tf.data.AUTOTUNE)
val_pipeline = val_pipeline.prefetch(buffer_size=tf.data.AUTOTUNE)

# Predict and collect results

In [ ]:
# --- 3. ทำนายผล (Predict) และรวบรวมผลลัพธ์ ---

y_pred = []
y_true = []
iteration = 0
total_batches = len(val_pipeline)

print(f"\nกำลังทำนายผลจากข้อมูล Validation ทั้งหมด {total_batches} batches...")
start_time = time.time()

for images, labels in val_pipeline:
    predictions = model.predict(images, verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(labels.numpy(), axis=1)
    y_pred.extend(predicted_classes)
    y_true.extend(true_classes)
    
    iteration += 1
    if iteration % 50 == 0:
        print(f"  Processed {iteration}/{total_batches} batches...")

end_time = time.time()
print(f"ทำนายผลเสร็จสิ้นใน {end_time - start_time:.2f} วินาที")

# Calculate and display results

In [ ]:
# --- 4. คำนวณและแสดงผลลัพธ์ ---

y_pred = np.array(y_pred)
y_true = np.array(y_true)

print("\n" + "="*70)
print("            ผลการประเมินโมเดล (Classification Report)          ")
print("="*70 + "\n")

present_labels = np.unique(y_true)
print(f"พบข้อมูลจริง {len(present_labels)} คลาสในชุด Validation ที่ใช้ประเมินผล")

present_class_names = [class_names[i] for i in present_labels]

report = classification_report(
    y_true, 
    y_pred, 
    labels=present_labels, 
    target_names=present_class_names,
    zero_division=0,
    digits=4
)
print(report)

print("\n--- สรุปค่าเฉลี่ยประสิทธิภาพโมเดล ---")
report_dict = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

print("\n[ แนะนำ ] ค่าเฉลี่ยแบบถ่วงน้ำหนัก (Weighted Average)")
print(f"  - Precision: {report_dict['weighted avg']['precision']:.4f}")
print(f"  - Recall:    {report_dict['weighted avg']['recall']:.4f}")
print(f"  - F1-Score:  {report_dict['weighted avg']['f1-score']:.4f}")

print("\nค่าเฉลี่ยแบบไม่ถ่วงน้ำหนัก (Macro Average)")
print(f"  - Precision: {report_dict['macro avg']['precision']:.4f}")
print(f"  - Recall:    {report_dict['macro avg']['recall']:.4f}")
print(f"  - F1-Score:  {report_dict['macro avg']['f1-score']:.4f}")

print("\nความแม่นยำโดยรวม (Overall Accuracy)")
print(f"  - Accuracy: {report_dict['accuracy']:.4f}")
print("="*40)

กำลังโหลดโมเดลจาก: /media/capybara/Data/dataset_vit/resnet50/best_resnet_model_imagenet2012v2_classes.keras
โหลดโมเดลสำเร็จ
กำลังโหลดชุดข้อมูล Validation จาก: /media/capybara/Data/dataset_vit/archive
Found 1281167 files belonging to 1000 classes.
Using 128116 files for validation.
พบ 1000 คลาส

กำลังทำนายผลจากข้อมูล Validation ทั้งหมด 4004 batches...
  Processed 50/4004 batches...
  Processed 100/4004 batches...
  Processed 150/4004 batches...
  Processed 200/4004 batches...
  Processed 250/4004 batches...
  Processed 300/4004 batches...
  Processed 350/4004 batches...
  Processed 400/4004 batches...
  Processed 450/4004 batches...
  Processed 500/4004 batches...
  Processed 550/4004 batches...
  Processed 600/4004 batches...
  Processed 650/4004 batches...
  Processed 700/4004 batches...
  Processed 750/4004 batches...
  Processed 800/4004 batches...
  Processed 850/4004 batches...
  Processed 900/4004 batches...
  Processed 950/4004 batches...
  Processed 1000/4004 batches...
  Proce

: 